## Chunking and Embedding

In [28]:
import uuid
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from pydantic import BaseModel
from langchain.chains import create_extraction_chain_pydantic
from dotenv import load_dotenv
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from typing import Optional, List
from langchain import hub

In [40]:
load_dotenv()

google_api_key = os.getenv("GEMINI_API_KEY")


class AgenticChunker:
    def __init__(self, google_api_key):
        self.chunks = {}
        self.id_truncate_limit = 5

        # Whether or not to update/refine summaries and titles as you get new information
        self.generate_new_metadata_ind = True
        self.print_logging = True

        if google_api_key is None:
            google_api_key = os.getenv("GEMINI_API_KEY")

        if google_api_key is None:
            raise ValueError("API key is not provided and not found in environment variables")

        # Use Gemini 1.5 Pro model
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash-latest",
            google_api_key=google_api_key,
            temperature=0
        )

    def add_propositions(self, propositions):
        for proposition in propositions:
            self.add_proposition(proposition)
    
    def add_proposition(self, proposition):
        if self.print_logging:
            print (f"\nAdding: '{proposition}'")

        # If it's your first chunk, just make a new chunk and don't check for others
        if len(self.chunks) == 0:
            if self.print_logging:
                print ("No chunks, creating a new one")
            self._create_new_chunk(proposition)
            return

        chunk_id = self._find_relevant_chunk(proposition)

        # If a chunk was found then add the proposition to it
        if chunk_id:
            if self.print_logging:
                print (f"Chunk Found ({self.chunks[chunk_id]['chunk_id']}), adding to: {self.chunks[chunk_id]['title']}")
            self.add_proposition_to_chunk(chunk_id, proposition)
            return
        else:
            if self.print_logging:
                print ("No chunks found")
            # If a chunk wasn't found, then create a new one
            self._create_new_chunk(proposition)
        

    def add_proposition_to_chunk(self, chunk_id, proposition):
        # Add then
        self.chunks[chunk_id]['propositions'].append(proposition)

        # Then grab a new summary
        if self.generate_new_metadata_ind:
            self.chunks[chunk_id]['summary'] = self._update_chunk_summary(self.chunks[chunk_id])
            self.chunks[chunk_id]['title'] = self._update_chunk_title(self.chunks[chunk_id])

    def _update_chunk_summary(self, chunk):
        """
        If you add a new proposition to a chunk, you may want to update the summary or else they could get stale
        """
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic
                    A new proposition was just added to one of your chunks, you should generate a very brief 1-sentence summary which will inform viewers what a chunk group is about.

                    A good summary will say what the chunk is about, and give any clarifying instructions on what to add to the chunk.

                    You will be given a group of propositions which are in the chunk and the chunks current summary.

                    Your summaries should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or month, generalize it to "date and times".

                    Example:
                    Input: Proposition: Greg likes to eat pizza
                    Output: This chunk contains information about the types of food Greg likes to eat.

                    Only respond with the chunk new summary, nothing else.
                    """,
                ),
                ("user", "Chunk's propositions:\n{proposition}\n\nCurrent chunk summary:\n{current_summary}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_summary = runnable.invoke({
            "proposition": "\n".join(chunk['propositions']),
            "current_summary" : chunk['summary']
        }).content

        return new_chunk_summary
    
    def _update_chunk_title(self, chunk):
        """
        If you add a new proposition to a chunk, you may want to update the title or else it can get stale
        """
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic
                    A new proposition was just added to one of your chunks, you should generate a very brief updated chunk title which will inform viewers what a chunk group is about.

                    A good title will say what the chunk is about.

                    You will be given a group of propositions which are in the chunk, chunk summary and the chunk title.

                    Your title should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or month, generalize it to "date and times".

                    Example:
                    Input: Summary: This chunk is about dates and times that the author talks about
                    Output: Date & Times

                    Only respond with the new chunk title, nothing else.
                    """,
                ),
                ("user", "Chunk's propositions:\n{proposition}\n\nChunk summary:\n{current_summary}\n\nCurrent chunk title:\n{current_title}"),
            ]
        )

        runnable = PROMPT | self.llm

        updated_chunk_title = runnable.invoke({
            "proposition": "\n".join(chunk['propositions']),
            "current_summary" : chunk['summary'],
            "current_title" : chunk['title']
        }).content

        return updated_chunk_title

    def _get_new_chunk_summary(self, proposition):
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic
                    You should generate a very brief 1-sentence summary which will inform viewers what a chunk group is about.

                    A good summary will say what the chunk is about, and give any clarifying instructions on what to add to the chunk.

                    You will be given a proposition which will go into a new chunk. This new chunk needs a summary.

                    Your summaries should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or month, generalize it to "date and times".

                    Example:
                    Input: Proposition: Greg likes to eat pizza
                    Output: This chunk contains information about the types of food Greg likes to eat.

                    Only respond with the new chunk summary, nothing else.
                    """,
                ),
                ("user", "Determine the summary of the new chunk that this proposition will go into:\n{proposition}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_summary = runnable.invoke({
            "proposition": proposition
        }).content

        return new_chunk_summary
    
    def _get_new_chunk_title(self, summary):
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic
                    You should generate a very brief few word chunk title which will inform viewers what a chunk group is about.

                    A good chunk title is brief but encompasses what the chunk is about

                    You will be given a summary of a chunk which needs a title

                    Your titles should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or month, generalize it to "date and times".

                    Example:
                    Input: Summary: This chunk is about dates and times that the author talks about
                    Output: Date & Times

                    Only respond with the new chunk title, nothing else.
                    """,
                ),
                ("user", "Determine the title of the chunk that this summary belongs to:\n{summary}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_title = runnable.invoke({
            "summary": summary
        }).content

        return new_chunk_title


    def _create_new_chunk(self, proposition):
        new_chunk_id = str(uuid.uuid4())[:self.id_truncate_limit] # I don't want long ids
        new_chunk_summary = self._get_new_chunk_summary(proposition)
        new_chunk_title = self._get_new_chunk_title(new_chunk_summary)

        self.chunks[new_chunk_id] = {
            'chunk_id' : new_chunk_id,
            'propositions': [proposition],
            'title' : new_chunk_title,
            'summary': new_chunk_summary,
            'chunk_index' : len(self.chunks)
        }
        if self.print_logging:
            print (f"Created new chunk ({new_chunk_id}): {new_chunk_title}")
    
    def get_chunk_outline(self):
        """
        Get a string which represents the chunks you currently have.
        This will be empty when you first start off
        """
        chunk_outline = ""

        for chunk_id, chunk in self.chunks.items():
            single_chunk_string = f"""Chunk ID: {chunk['chunk_id']}\nChunk Name: {chunk['title']}\nChunk Summary: {chunk['summary']}\n\n"""
        
            chunk_outline += single_chunk_string
        
        return chunk_outline

    def _find_relevant_chunk(self, proposition):
        current_chunk_outline = self.get_chunk_outline()

        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    Determine whether or not the "Proposition" should belong to any of the existing chunks.

                    A proposition should belong to a chunk of their meaning, direction, or intention are similar.
                    The goal is to group similar propositions and chunks.

                    If you think a proposition should be joined with a chunk, return the chunk id.
                    If you do not think an item should be joined with an existing chunk, just return "No chunks"

                    Example:
                    Input:
                        - Proposition: "Greg really likes hamburgers"
                        - Current Chunks:
                            - Chunk ID: 2n4l3d
                            - Chunk Name: Places in San Francisco
                            - Chunk Summary: Overview of the things to do with San Francisco Places

                            - Chunk ID: 93833k
                            - Chunk Name: Food Greg likes
                            - Chunk Summary: Lists of the food and dishes that Greg likes
                    Output: 93833k
                    """,
                ),
                ("user", "Current Chunks:\n--Start of current chunks--\n{current_chunk_outline}\n--End of current chunks--"),
                ("user", "Determine if the following statement should belong to one of the chunks outlined:\n{proposition}"),
            ]
        )

        runnable = PROMPT | self.llm

        chunk_found = runnable.invoke({
            "proposition": proposition,
            "current_chunk_outline": current_chunk_outline
        }).content

        # Pydantic data class
        class ChunkID(BaseModel):
            """Extracting the chunk id"""
            chunk_id: Optional[str]
            
        # Extraction to catch-all LLM responses. This is a bandaid
        extraction_chain = create_extraction_chain_pydantic(pydantic_schema=ChunkID, llm=self.llm)
        extraction_found = extraction_chain.run(chunk_found)
        if extraction_found:
            chunk_found = extraction_found[0].chunk_id

        # If you got a response that isn't the chunk id limit, chances are it's a bad response or it found nothing
        # So return nothing
        if len(chunk_found) != self.id_truncate_limit:
            return None

        return chunk_found
    
    def get_chunks(self, get_type='dict'):
        """
        This function returns the chunks in the format specified by the 'get_type' parameter.
        If 'get_type' is 'dict', it returns the chunks as a dictionary.
        If 'get_type' is 'list_of_strings', it returns the chunks as a list of strings, where each string is a proposition in the chunk.
        """
        if get_type == 'dict':
            return self.chunks
        if get_type == 'list_of_strings':
            chunks = []
            for chunk_id, chunk in self.chunks.items():
                chunks.append(" ".join([x for x in chunk['propositions']]))
            return chunks
    
    def pretty_print_chunks(self):
        print (f"\nYou have {len(self.chunks)} chunks\n")
        for chunk_id, chunk in self.chunks.items():
            print(f"Chunk #{chunk['chunk_index']}")
            print(f"Chunk ID: {chunk_id}")
            print(f"Summary: {chunk['summary']}")
            print(f"Propositions:")
            for prop in chunk['propositions']:
                print(f"    -{prop}")
            print("\n\n")

    def pretty_print_chunk_outline(self):
        print ("Chunk Outline\n")
        print(self.get_chunk_outline())


In [21]:
import os
import getpass

os.environ["NVIDIA_API_KEY"] = getpass.getpass("Provide your NVIDIA API key: ")

Provide your NVIDIA API key: ········


In [8]:
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Provide your LANGSMITH API key: ")

Provide your LANGSMITH API key: ········


In [25]:
os.environ["GEMINI_API_KEY"] = getpass.getpass("Provide your GEMINI API key: ")

Provide your GEMINI API key: ········


In [44]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Provide your COHERE API key: ")

Provide your COHERE API key: ········


In [50]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
# from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_cohere import CohereEmbeddings

client = QdrantClient(":memory:")

client.create_collection(
    collection_name = "study_data_collection",
    vectors_config = VectorParams(size = 1024, distance = Distance.COSINE),
)

cohere_embedding = CohereEmbeddings(
    model="embed-english-v3.0",
    cohere_api_key=os.getenv("COHERE_API_KEY")
)


In [46]:
# !pip install langchain_cohere

In [41]:

obj = hub.pull("wfh/proposal-indexing")
llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash-latest",
            google_api_key=google_api_key,
            temperature=0
        )
runnable = obj | llm


class Sentences(BaseModel):
    sentences: List[str]
    

from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """Extract the key propositions from the following text. 
List each proposition as a bullet point.

Text:
{text}
    
Propositions:"""
)

def get_propositions(text):
    final_prompt = prompt.format(text=text)
    result = llm.invoke(final_prompt).content

    # Parse the bullet list into a clean list of strings
    lines = result.strip().split("\n")
    propositions = [line.lstrip("-• ").strip() for line in lines if line.strip()]
    return propositions


C:\Users\sudda\anaconda3\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
# !pip install langchain_ollama

In [51]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document


ac = AgenticChunker(google_api_key)

def read_pdf(path):
    loader = PyPDFLoader(path)
    file = loader.load()
    
    text = "\n".join([doc.page_content for doc in file])
    paragraphs = text.split("\n\n")
    
    return paragraphs

def encode_pdf(path) :
    paragraphs = read_pdf(path)
    
    propositions = get_propositions(paragraphs)
    
    ac.add_proposition(propositions)
    
    print(propositions)

def embed_in_vector_db(ac,client,cohere_embedding,collection_name = "study_data_collection"):
    documents = []
    
    for chunk in ac.get_chunks(get_type='dict').values():
        content = "\n".join([p for sublist in chunk['propositions'] for p in (sublist if isinstance(sublist, list) else [sublist])])

        metadata = {
            "chunk_id": chunk['chunk_id'],
            "title": chunk['title'],
            "summary": chunk['summary'],
            "chunk_index": chunk['chunk_index']
        }
        
        documents.append(Document(page_content=content,metadata=metadata))
        
    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=cohere_embedding,
    )
    
    vectorstore.add_documents(documents)
    
    return vectorstore
    
    
    
# path = input()

# encode_pdf(path)

# embed_in_vector_db(ac,client,cohere_embedding,"study_data_collection")


C:\Users\sudda\Downloads\Shatadeep_resume (3).pdf

Adding: '['* Shatadeep De is a 3rd-year undergraduate student of Computer Science and Engineering at IIT Bhubaneswar.', '* His CGPA is 9.08/10.', '* He previously studied Civil Engineering at IIT Bhubaneswar.', '* He achieved 12th rank statewide in the West Bengal Council of Higher Secondary Education (WBCHSE) Class 12th Board Examination.', '* He achieved a 9.53 CGPA in his first year of university, leading to a branch change from Civil Engineering to Computer Science and Engineering.', '* He developed a Retrieval-Augmented Generation (RAG) system for precise, context-aware answers from .txt documents.', '* He developed a tool to track and visualize personal competitive programming performance using the Codeforces API.', '* He developed a personal utility to monitor YouTube watch time and maintain a daily learning streak.', '* He built a Nearby Hospital Locator CLI project using C.', '* He possesses skills in various programming langu

In [56]:
# from qdrant_client.http.models import CountRequest

# # Check how many points/documents are stored
# count = client.count(
#     collection_name="study_data_collection",
#     exact = True,
# )

# print(f"Number of vectors stored: {count.count}")


In [57]:
# from qdrant_client.http.models import Filter

# client.delete(
#     collection_name="study_data_collection",
#     points_selector=Filter(must=[])  # matches all points
# )
